# Regression Used Cars Dataset

https://www.kaggle.com/datasets/austinreese/craigslist-carstrucks-data 


#### **Dependency**

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from itertools import combinations  # For creating combinations of elements
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans

#### **Utility Function**

In [4]:
def reduce_mem_usage(df, verbose=0):
    """
    Iterate through all numeric columns of a dataframe and modify the data type
    to reduce memory usage.
    """
    # Calculate the initial memory usage of the DataFrame
    start_mem = df.memory_usage().sum() / 1024**2

    # 🔄 Iterate through each column in the DataFrame
    for col in df.columns:
        col_type = df[col].dtype
        # Check if the column's data type is not 'object' (i.e., numeric)
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            # Check if the column's data type is an integer
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                # Check if the column's data type is a float
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)
    # ℹ️ Provide memory optimization information if 'verbose' is True
    if verbose:
        print(f"Memory usage of dataframe is {start_mem:.2f} MB")
        end_mem = df.memory_usage().sum() / 1024**2
        print(f"Memory usage after optimization is: {end_mem:.2f} MB")
        decrease = 100 * (start_mem - end_mem) / start_mem
        print(f"Decreased by {decrease:.2f}%")

    # Return the DataFrame with optimized memory usage
    return df

#### **Load Data**

load csv inside data folder

In [7]:
df = pd.read_csv("kaggle/input/vehicles/vehicles.csv")
df=reduce_mem_usage(df,verbose=True)
df.shape
df[0:4]

Memory usage of dataframe is 84.68 MB
Memory usage after optimization is: 76.54 MB
Decreased by 9.62%


,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN


#### **Split dataset** 

In [14]:
df_train, df_test = train_test_split(df, test_size=0.2)
print(df_train.shape, df_test.shape)

(341504, 26) (85376, 26)


#### **Data Function**

data preperation

feature engineering

In [15]:
# implement the function
def data_preparation(df:pd.DataFrame)->pd.DataFrame:
    """"
    make data valid
    """
    #Create df1, get the average value of deleting NAN and outliers, then assign it to the NAN in df, 
    #and then delete the outliers in df to get the latest data set
    df1 = df.copy()
    odometer_data = df1['odometer']
    odometer_data_cleaned = odometer_data.dropna()
    kmeans = KMeans(n_clusters=4)
    kmeans.fit(odometer_data_cleaned.values.reshape(-1, 1))

    outliers = odometer_data_cleaned[kmeans.labels_ == 1]
    df1 = df1[~df1['odometer'].isin(outliers)]
    average_value = df1['odometer'].mean()
    df['odometer'] = df['odometer'].fillna(average_value)
    
    #Use Kmeans to calculate outliers and delete them together with data with a price less than 1,000
    #($1,000 is about ￥7,500. Even the price of a second-hand car is too low, so it is also regarded as an outlier and deleted)
    price_data = df['price']
    kmeans = KMeans(n_clusters=2)
    kmeans.fit(price_data.values.reshape(-1, 1))
    outliers = price_data[kmeans.labels_ == 1]

    df = df[~((price_data.isin(outliers)) | (price_data < 1000))]
    
    #use linear regression to prediect values for null in year col.
    features = ["price",'odometer']
    df_complete = df.dropna(subset=["year"] + features)
    X_train = df_complete[features]
    y_train = df_complete["year"]
    regression_model_year = LinearRegression()
    regression_model_year.fit(X_train, y_train)
    X_missing = df[df["year"].isnull()][features]
    predicted_year = regression_model_year.predict(X_missing)

    df.loc[df["year"].isnull(), "year"] = predicted_year.round().astype(int)
    
    #Fill null values using the modes of "manufacturer", "cylinders", 'fuel',"title_status", "transmission","drive", "type", "paint_color", "lat", "long"
    columns_to_fillna = ['manufacturer', 'cylinders', 'fuel', 'title_status', 'transmission',
                     'drive', 'type', 'paint_color', 'lat', 'long']
    modes = df[columns_to_fillna].mode().iloc[0]
    df.loc[:, columns_to_fillna] = df.loc[:, columns_to_fillna].fillna(value=modes)
    
    #for "condition" col: Fill NAN with randomly selected data from that column to manipulate the more possible results.
    condition_options = df["condition"].unique()
    df.loc[df["condition"].isnull(), "condition"] = np.random.choice(condition_options)

    # turn nominal data into number
    y = df['price']
    x =df.drop(columns=['price'])


    return x,y

def feature_engineering(df:pd.DataFrame)->pd.DataFrame:
    """"
    select useful feature
    """
    #vehicle_age=posting_year - year, get the usage time of this vehicle
    df["posting_date"] = pd.to_datetime(df["posting_date"], utc=True)
    df["posting_year"] = df["posting_date"].dt.year
    df["vehicle_age"] = df["posting_year"] - df["year"]
    
    #Remove meaningless columns
    df.drop(['id','url','region','region_url','model','title_status',
         'VIN','size','image_url','description','county',
         'state','lat','long','posting_date','posting_year'], 
        axis=1, inplace=True)
    
    return df

def data_preprocessing(df:pd.DataFrame)->pd.DataFrame:
    x,y =data_preparation(df)
    x_pre=feature_engineering(x)
    return x_pre,y


#### RUN Preprocess

In [19]:
df_train_x,df_train_y = data_preprocessing(df_train)
df_test_x, df_test_y = data_preprocessing(df_test)

print(df_train_x.shape, df_train_y.shape)
df_train_x[0:10]
# df_train_y[0:10]

(304436, 11) (304436,)


,year,manufacturer,condition,cylinders,fuel,odometer,transmission,drive,type,paint_color,vehicle_age
36736,2012.0,ford,like new,6 cylinders,gas,85970.0,automatic,4wd,sedan,white,9.0
418384,2009.0,chevrolet,like new,4 cylinders,gas,170309.0,manual,fwd,coupe,white,12.0
393520,2012.0,kia,good,4 cylinders,gas,151000.0,automatic,fwd,sedan,white,9.0
211571,2007.0,chevrolet,excellent,6 cylinders,diesel,113941.0,automatic,4wd,pickup,green,14.0
252407,2002.0,lexus,fair,6 cylinders,gas,215000.0,automatic,fwd,coupe,black,19.0
151056,2012.0,toyota,excellent,6 cylinders,gas,108526.0,automatic,fwd,mini-van,white,9.0
26408,2014.0,ford,good,6 cylinders,gas,95330.0,automatic,4wd,truck,black,7.0
187918,1992.0,jaguar,like new,6 cylinders,gas,100000.0,automatic,4wd,convertible,blue,29.0
235843,2020.0,honda,good,6 cylinders,gas,9954.0,other,fwd,coupe,silver,1.0
248297,2013.0,ford,like new,6 cylinders,gas,70000.0,automatic,4wd,sedan,white,8.0


#### **Training and Evaluation**

In [ ]:
numJobs=3
cv=5
paramgrid={}

model = LinearRegression().fit(df_train_x, df_train_y)
#cross validation 
modelCV= GridSearchCV(model,paramgrid,n_jobs=numJobs,cv=cv)

#predict
y_pred = modelCV.predict(df_test_x)

#evaluation

# RMSE
rmse = np.sqrt(mean_squared_error(df_test_y, y_pred))
print(f"Root Mean Squared Error: {rmse}")

# MAE
mae = mean_absolute_error(df_test_y, y_pred)
print(f"Mean Absolute Error: {mae}")

# R2
r2 = r2_score(df_test_y, y_pred)
print(f"R-squared: {r2}")

#Residual Analysis: 
residuals = df_test_y - y_pred

plt.scatter(y_pred, residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.show()

### Trian and Evaluation By Hessia

In [23]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
#label encoding

encoder = LabelEncoder()
columns_to_encode = ['manufacturer', 'condition','cylinders','fuel','transmission','drive','type','paint_color']
df_train_x[columns_to_encode] = df_train_x[columns_to_encode].apply(lambda x: encoder.fit_transform(x))
df_test_x[columns_to_encode] = df_test_x[columns_to_encode].apply(lambda x: encoder.fit_transform(x))
# df_train_x[0:10]

#normalization
scaler = StandardScaler()
train_x_normalized_data = scaler.fit_transform(df_train_x)
test_x_normalized_data = scaler.fit_transform(df_test_x)
# train_x_normalized_data[0:10]

reg = GradientBoostingRegressor( subsample=1.0, criterion="friedman_mse", min_samples_split=2,
                                min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3,
                                min_impurity_decrease=0.0, init=None,
                                random_state=None, max_features=None, alpha=0.9, verbose=0,
                                max_leaf_nodes=None, warm_start=False,
                                validation_fraction=0.1, n_iter_no_change=None, tol=0.0001)
reg.fit(train_x_normalized_data, df_train_y)
reg.score(test_x_normalized_data, df_test_y)


ValueError: Input X contains NaN.
GradientBoostingRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values